## 1. Setup & Imports

In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

## 2. Load Preprocessed Data

In [2]:
X_train = pd.read_csv("D:/Project/data/X_train.csv")
X_test = pd.read_csv("D:/Project/data/X_test.csv")
y_train = pd.read_csv("D:/Project/data/y_train.csv")
y_test = pd.read_csv("D:/Project/data/y_test.csv")

## 3. Load Model and Scaler

In [3]:
# Loading saved Data
loaded_model = joblib.load('D:/Project/model/logistic_model.pkl')
loaded_scaler = joblib.load('D:/Project/model/scaler.pkl')

# Applying scaler to numeric columns
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges', 'NumServicesUsed']
X_test[num_cols] = loaded_scaler.transform(X_test[num_cols])

# Checking Model
preds = loaded_model.predict(X_test)
print(preds)

[1 1 0 ... 0 0 0]


## 4. Evaluate Model

In [4]:
print(classification_report(y_test,preds))
print("ROC AUC Score:",roc_auc_score(y_test,loaded_model.predict_proba(X_test)[:,1]))

              precision    recall  f1-score   support

           0       0.95      0.29      0.44      1033
           1       0.33      0.96      0.49       374

    accuracy                           0.47      1407
   macro avg       0.64      0.62      0.46      1407
weighted avg       0.79      0.47      0.45      1407

ROC AUC Score: 0.7691747208431907


## 5. Saving Prediction Example

In [5]:
sample = X_test.iloc[[0]].copy()
sample[num_cols] = loaded_scaler.transform(sample[num_cols])

predicted_class = loaded_model.predict(sample)[0]
print("Predicted Class:", predicted_class)

Predicted Class: 1


In [6]:
proba = loaded_model.predict_proba(sample)[0]

print(f"Predicted Class: {'Churn' if predicted_class == 1 else 'No Churn'}")
print(f"Probability of No Churn: {proba[0]:.2f}")
print(f"Probability of Churn:    {proba[1]:.2f}")

Predicted Class: Churn
Probability of No Churn: 0.45
Probability of Churn:    0.55


In [7]:
def predict_customer_churn(sample, model, scaler, num_cols):
    """
    Predict churn for a single customer sample.

    Parameters:
        sample (DataFrame): Single-row DataFrame with customer data
        model (sklearn model): Trained classification model
        scaler (StandardScaler): Fitted scaler for numeric features
        num_cols (list): List of numeric columns that require scaling

    Returns:
        dict: Dictionary with predicted class and probabilities
    """
    # Make a copy to avoid modifying original input
    sample_copy = sample.copy()
    
    # Scale numeric columns
    sample_copy[num_cols] = scaler.transform(sample_copy[num_cols])
    
    # Predict class and probabilities
    pred_class = model.predict(sample_copy)[0]
    proba = model.predict_proba(sample_copy)[0]

    return {
        "predicted_class": "Churn" if pred_class == 1 else "No Churn",
        "probability_churn": round(proba[1], 2),
        "probability_no_churn": round(proba[0], 2)
    }


In [8]:
# Define numeric columns
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges', 'NumServicesUsed']

# Pick a sample
sample = X_test.iloc[[0]]

# Call the function
result = predict_customer_churn(sample, loaded_model, loaded_scaler, num_cols)

# Show the result
print(result)


{'predicted_class': 'Churn', 'probability_churn': 0.55, 'probability_no_churn': 0.45}
